# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [38]:
# import modules
from typing import List, Optional, Dict

**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [39]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [40]:
# This is old code but i do not want to delete it yet xoxox sjon
# set up dimacs rules sudoku
with open('./sudoku-rules-9x9.txt') as f:
    rawRules = f.read().splitlines()

# Rules is een lijstje met clauses met individuele variabelen
rules: list[list[int]] = []
for clause in rawRules[1:]:
    rules.append(list(map(int, clause.split()))[:-1])

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [41]:
def load_rules(filename: str) -> List[List[int]]:
    with open(filename) as f:
        raw_rules = f.read().splitlines()

    rules = [list(map(int, clause.split()))[:-1] for clause in raw_rules[1:]]
    return rules

def single_literal(clauses: List[List[int]], assignment: Dict[int, bool]) -> Optional[List[List[int]]]:
    """
    If a clause has only one literal, assigns it to satisfy the clause, removes satisfied clauses, 
    and removes negated literals from the remaining clauses. Repeats until no single-literal 
    clauses remain or an unsatisfiable clause (empty clause) is found.

    Args:
        clauses (List[List[int]]): List of clauses, each a list of integers (literals).
        assignment (Dict[int, bool]): Current variable assignments, aka the sudoku.

    Returns:
        Optional[List[List[int]]]: Updated list of clauses, or None if unsatisfiable.
    """
    while any(len(clause) == 1 for clause in clauses):
        for clause in clauses:
            if len(clause) == 1: #single literal clause
                literal = clause[0]
                assignment[abs(literal)] = literal > 0  # Assign truth value
                
                # update clauses
                new_clauses = []
                for c in clauses:
                    if literal in c:  # clause satisfied, skip it
                        continue
                    updated_clause = [l for l in c if l != -literal] # check for empty or conflicting
                    if not updated_clause:  # empty clause, unsatisfiable
                        return None
                    new_clauses.append(updated_clause)
                clauses = new_clauses
                break
    return clauses


def pure_literal(clauses: List[List[int]], assignment: Dict[int, bool]) -> List[List[int]]:
    """
    Assigns values to pure literals (literals with no negation in any clause) and updates clauses.

    Args:
        clauses (List[List[int]]): List of clauses, each as a list of literals.
        assignment (Dict[int, bool]): Dictionary of current variable assignments, aka the sudoku.

    Returns:
        List[List[int]]: Updated list of clauses with pure literals removed.
    """
    literals = {lit for clause in clauses for lit in clause}
    pure_literals = [l for l in literals if -l not in literals]
    
    for literal in pure_literals:
        assignment[abs(literal)] = literal > 0 # assign T/F
        clauses = [clause for clause in clauses if literal not in clause]
    
    return clauses



**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [42]:
def dpll(clauses: List[List[int]], assignment: Dict[int, bool]) -> Optional[Dict[int, bool]]:
    """
    Solves a set of clauses using the DPLL algorithm.

    Args:
        clauses (List[List[int]]): A list of clauses, each as a list of literals.
        assignment (Dict[int, bool]): Current variable assignments, aka the sudoku.

    Returns:
        Optional[Dict[int, bool]]: Satisfying assignment if one exists, otherwise None.
    """
    clauses = single_literal(clauses, assignment)
    if clauses is None:
        return None  # conflict
    if not clauses:
        return assignment  # all clauses satisfied

    clauses = pure_literal(clauses, assignment)
    if not clauses:
        return assignment  # all clauses satisfied

    # choose a literal from the first clause
    literal = clauses[0][0]
    var = abs(literal)

    # Recursive DPLL call
    assignment[var] = True
    if dpll([c for c in clauses if literal not in c], assignment.copy()): # if positive literal
        return assignment

    assignment[var] = False
    if dpll([c for c in clauses if -literal not in c], assignment.copy()): # if negative literal
        return assignment

    return None

*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**Heuristics** - Two different heuristics of our choice

In [43]:
# heuristic one

In [44]:
# heuristic two

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [45]:
def solve_sudoku(rules: List[List[int]], puzzle: List[List[int]]) -> Optional[List[List[int]]]:
    """
    Solves a Sudoku puzzle using given rules and an initial puzzle setup.

    Args:
        rules (List[List[int]]): A list of rules in CNF form for Sudoku constraints.
        puzzle (List[List[int]]): A 9x9 grid representing the puzzle, with `0` for empty cells.

    Returns:
        Optional[List[List[int]]]: A solved 9x9 Sudoku grid if a solution exists, otherwise None.
    """
    assignment = {} # the sudoku
    for i, row in enumerate(puzzle):
        for j, value in enumerate(row):
            if value != 0:  # if 0, cell is empty, if not 0, it is a pre-filled cell
                assignment[(i + 1) * 100 + (j + 1) * 10 + value] = True

    solution = dpll(rules, assignment)
    if not solution:
        return None

    # parse the solution back to a 9x9 grid
    result = [[0] * 9 for _ in range(9)]
    for var, value in solution.items():
        if value:
            i = (var // 100) - 1
            j = ((var // 10) % 10) - 1
            num = var % 10
            result[i][j] = num
    return result

**Test Sudoku** - Try and see if it works

In [47]:
filename = './sudoku-rules-9x9.txt'
rules = load_rules(filename)
print(rules)
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]
solution = solve_sudoku(rules, puzzle)

if solution:
    for row in solution:
        print(row)
else:
    print("No solution found")

[[111, 112, 113, 114, 115, 116, 117, 118, 119], [-111, -112], [-111, -113], [-111, -114], [-111, -115], [-111, -116], [-111, -117], [-111, -118], [-111, -119], [-112, -113], [-112, -114], [-112, -115], [-112, -116], [-112, -117], [-112, -118], [-112, -119], [-113, -114], [-113, -115], [-113, -116], [-113, -117], [-113, -118], [-113, -119], [-114, -115], [-114, -116], [-114, -117], [-114, -118], [-114, -119], [-115, -116], [-115, -117], [-115, -118], [-115, -119], [-116, -117], [-116, -118], [-116, -119], [-117, -118], [-117, -119], [-118, -119], [121, 122, 123, 124, 125, 126, 127, 128, 129], [-121, -122], [-121, -123], [-121, -124], [-121, -125], [-121, -126], [-121, -127], [-121, -128], [-121, -129], [-122, -123], [-122, -124], [-122, -125], [-122, -126], [-122, -127], [-122, -128], [-122, -129], [-123, -124], [-123, -125], [-123, -126], [-123, -127], [-123, -128], [-123, -129], [-124, -125], [-124, -126], [-124, -127], [-124, -128], [-124, -129], [-125, -126], [-125, -127], [-125, -1

****
## Part 2 - Experiment 

**Optional Hypothesis**: "Advancements in SAT solver techniques, tested and optimized using Sudoku puzzles as a structured benchmark, can significantly improve the efficiency of scheduling and resource allocation tasks in complex real-world applications, such as airline crew scheduling and automated exam timetabling. Specifically, Sudoku-based testing will highlight solvers' ability to handle strict mutual exclusivity and completeness constraints, which are essential for minimizing conflicts and optimizing resource usage in these applications."

This hypothesis aims to leverage Sudoku as a standardized problem to assess the performance and flexibility of SAT solvers in environments requiring rapid, constraint-based decision-making, potentially providing insights into solver improvements for more complex, real-world scheduling and resource optimization scenarios.